In [1]:
import os
import re
import tabula
import pandas as pd
import numpy as np

In [2]:
# Set the JAVA_HOME environment variable to the Java installation directory
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk/libexec/openjdk.jdk"

In [3]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [6]:
%pip install -q tabula-py
%pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### **Importing**

In [5]:
# Only Jan 2018 to Nov 2022 is machine readable
# Criteria for downloading: more than 2 nbsp and href link contains nebula
# tabula.read_pdf(... , pages=all)

In [55]:
import os
import requests
from bs4 import BeautifulSoup

# URL to be scraped
url = "https://www.akelco.com.ph/rates.html"

# Make a GET request to fetch the raw HTML content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find all strong tags
    strong_tags = soup.find_all('strong')
    
    base_url = "http://nebula.wsimg.com"
    
    def is_valid_pdf_link(tag):
        # Check for the 'a' tag directly under 'strong' or nested within it
        link_tag = tag.find_parent('a') or tag.find('a')
        if link_tag and '-' in tag.get_text() and 'href' in link_tag.attrs and 'nebula' in link_tag['href']:
            return True
        return False
    
    # List to store valid PDF links and their corresponding names
    pdf_links = []
    
    # Iterate over strong tags and check if they meet the criteria
    for strong in strong_tags:
        link_tag = strong.find_parent('a') or strong.find('a')
        if link_tag and is_valid_pdf_link(strong):
            text = ' '.join(strong.get_text().split())  # Remove unnecessary spaces
            pdf_links.append((link_tag['href'], text))
    
    # Download the PDFs
    download_directory = 'pdf downloads'
    os.makedirs(download_directory, exist_ok=True)
    
    for link, text in pdf_links:
        full_url = link if link.startswith('http') else base_url + link
        response = requests.get(full_url)
        pdf_name = text + '.pdf'
        with open(os.path.join(download_directory, pdf_name), 'wb') as pdf_file:
            pdf_file.write(response.content)
            print(f'Downloaded: {pdf_name}')
    
    print('PDFs have been downloaded successfully.')
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


Downloaded: OCTOBER 2022 - NOVEMBER 2022.pdf
Downloaded: ​APRIL 2022 - SEPTEMBER 2022.pdf
Downloaded: NOVEMBER 2020 - MARCH 2022.pdf
Downloaded: ​JUNE 2020 - OCTOBER 2020.pdf
Downloaded: JANUARY 2018 - MAY 2020.pdf
PDFs have been downloaded successfully.
